# Initial tests for classification on the photoswitch dataset


In [40]:
%load_ext autoreload

%autoreload 2

In [32]:
from gpt3forchem.data import get_photoswitch_data
from gpt3forchem.input import create_single_property_forward_prompts
from sklearn.model_selection import train_test_split
from gpt3forchem.api_wrappers import fine_tune, query_gpt3, extract_prediction
import time
from pycm import ConfusionMatrix


In [2]:
data = get_photoswitch_data()


In [7]:
prompts = create_single_property_forward_prompts(
    data,
    "wavelength_cat",
    {"wavelength_cat": "transition wavelength"},
    representation_col="SMILES",
)


In [8]:
train_prompts, test_prompts = train_test_split(
    prompts, test_size=0.2, random_state=None
)


In [10]:
train_prompts


,prompt,completion
3,what is the transition wavelength of C[N]1C=C(...,0@@@
332,what is the transition wavelength of Cc1[nH]c2...,1@@@
289,what is the transition wavelength of OCCN(CCO)...,1@@@
363,what is the transition wavelength of COC(C=C4)...,1@@@
247,what is the transition wavelength of CC(C=C(N(...,2@@@
...,...,...
135,what is the transition wavelength of CC1=CC=CC...,0@@@
50,what is the transition wavelength of O=C(COC1=...,0@@@
219,what is the transition wavelength of [H]C1=CC(...,3@@@
281,what is the transition wavelength of CC(O)C1=C...,1@@@


In [13]:
train_size = len(train_prompts)
test_size = len(test_prompts)

filename_base = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
train_filename = f"run_files/{filename_base}_train_prompts_polymers_{train_size}.jsonl"
valid_filename = f"run_files/{filename_base}_valid_prompts_polymers_{test_size}.jsonl"

train_prompts.to_json(train_filename, orient="records", lines=True)
test_prompts.to_json(valid_filename, orient="records", lines=True)


In [14]:
fine_tune(train_filename, valid_filename, "ada")


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.13.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.13.1
wandb: Run data is saved locally in /Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/wandb/run-20220831_112133-ft-ZTxafJhLHFoI8mc5JUlWiiWo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-ZTxafJhLHFoI8mc5JUlWiiWo
wandb: ⭐️ View project at https://wandb.ai/kjappelbaum/GPT-3
wandb: 🚀 View run at https://wandb.ai/kjappelbaum/GPT-3/runs/ft-ZTxafJhLHFoI8mc5JUlWiiWo
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:               elapsed_tokens ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:                training_loss 

🎉 wandb sync completed successfully


'ada:ft-lsmoepfl-2022-08-31-09-21-23'

In [16]:
completions = query_gpt3("ada:ft-lsmoepfl-2022-08-31-09-21-23", test_prompts)


In [31]:
predictions = [
    int(extract_prediction(completions, i)) for i in range(len(completions["choices"]))
]


In [35]:
true = [int(e.split("@")[0]) for e in test_prompts['completion']]

In [38]:
cm = ConfusionMatrix(true, predictions)

In [39]:
print(cm)

Predict  0        1        2        3        4        
Actual
0        21       5        0        0        0        

1        4        15       3        0        0        

2        0        3        20       0        0        

3        0        0        1        4        2        

4        0        0        0        1        0        





Overall Statistics : 

95% CI                                                            (0.66525,0.85374)
ACC Macro                                                         0.9038
ARI                                                               0.50551
AUNP                                                              0.83661
AUNU                                                              0.76037
Bangdiwala B                                                      0.62006
Bennett S                                                         0.69937
CBA                                                               0.57293
CSI                           